# Krigagem de temperatura, umidade e preciptação para todos os municípios de Minas Gerais no período de 2016 a 2018

Sofia Moreira de Aguiar | https://github.com/SofAguiar | sof.m.aguiar@gmail.com

Fonte de dados:
  - https://www.ibge.gov.br/geociencias/organizacao-do-territorio/malhas-territoriais/15774-malhas.html
  - https://portal.inmet.gov.br/paginas/catalogoaut
  - https://portal.inmet.gov.br/dadoshistoricos

### Setup do ambiente

In [ ]:
!pip install pykrige

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
from pykrige.ok import OrdinaryKriging
import os

### Upload e tratamento das bases

In [ ]:
# Utilizando a malha de municipios mais recente do IBGE para minas
url_malha = "https://geoftp.ibge.gov.br/organizacao_do_territorio/malhas_territoriais/malhas_municipais/municipio_2022/UFs/MG/MG_Municipios_2022.zip"

!wget -O MG_Municipios_2023.zip $url_malha

--2025-02-19 01:04:02--  https://geoftp.ibge.gov.br/organizacao_do_territorio/malhas_territoriais/malhas_municipais/municipio_2022/UFs/MG/MG_Municipios_2022.zip
Resolving geoftp.ibge.gov.br (geoftp.ibge.gov.br)... 191.233.136.86
Connecting to geoftp.ibge.gov.br (geoftp.ibge.gov.br)|191.233.136.86|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33017561 (31M) [application/zip]
Saving to: ‘MG_Municipios_2023.zip’

MG_Municipios_2023. 100%[===================>]  31.49M  12.7MB/s    in 2.5s    

2025-02-19 01:04:06 (12.7 MB/s) - ‘MG_Municipios_2023.zip’ saved [33017561/33017561]



In [ ]:
!unzip MG_Municipios_2023.zip -d "MG_Municipios_2023" # Descompactando a pasta com a malha municipal

Archive:  MG_Municipios_2023.zip
 extracting: MG_Municipios_2023/MG_Municipios_2022.cpg  
  inflating: MG_Municipios_2023/MG_Municipios_2022.dbf  
  inflating: MG_Municipios_2023/MG_Municipios_2022.prj  
  inflating: MG_Municipios_2023/MG_Municipios_2022.shp  
  inflating: MG_Municipios_2023/MG_Municipios_2022.shx  


In [ ]:
# Lendo shapefile
municipios = gpd.read_file('MG_Municipios_2023/MG_Municipios_2023.shp')

In [ ]:
# Utilizando os dados das estações previamente tratados
url_estacoes = "https://github.com/SofAguiar/krigagem_clima/raw/main/estacoes_mg.zip"

!wget -O estacoes_mg.zip $url_estacoes

--2025-02-19 01:11:20--  https://github.com/SofAguiar/krigagem_clima/raw/main/estacoes_mg.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/SofAguiar/krigagem_clima/main/estacoes_mg.zip [following]
--2025-02-19 01:11:21--  https://raw.githubusercontent.com/SofAguiar/krigagem_clima/main/estacoes_mg.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1509045 (1.4M) [application/zip]
Saving to: ‘estacoes_mg.zip’

estacoes_mg.zip     100%[===================>]   1.44M  --.-KB/s    in 0.07s   

2025-02-19 01:11:21 (20.5 MB/s) - ‘estacoes_mg.zip’ saved [1509045/1509045]



In [ ]:
!unzip estacoes_mg.zip -d "estacoes_mg" # Descompactando a pasta com os dados das estações

Archive:  estacoes_mg.zip
  inflating: estacoes_mg/estacoes_mg.csv  


In [ ]:
df_estacoes = pd.read_csv("estacoes_mg/estacoes_mg.csv")

In [ ]:
# Convertendo as colunas de data e variáveis para os tipos adequados
df_estacoes['Data'] = pd.to_datetime(df_estacoes['Data'], errors='coerce')
df_estacoes['temperatura_media'] = pd.to_numeric(df_estacoes['temperatura_media'], errors='coerce')
df_estacoes['preciptacao'] = pd.to_numeric(df_estacoes['preciptacao'], errors='coerce')
df_estacoes['umidade_media'] = pd.to_numeric(df_estacoes['umidade_media'], errors='coerce')
df_estacoes['VL_LATITUDE'] = pd.to_numeric(df_estacoes['VL_LATITUDE'], errors='coerce')
df_estacoes['VL_LONGITUDE'] = pd.to_numeric(df_estacoes['VL_LONGITUDE'], errors='coerce')


In [ ]:
# Garantindo que o CRS seja WGS84 (EPSG:4326)
municipios = municipios.to_crs(epsg=4326)

# Calculando os centróides dos municípios para usá-los como pontos de interpolação
municipios['centroide'] = municipios.geometry.centroid
municipios['lon'] = municipios['centroide'].x
municipios['lat'] = municipios['centroide'].y

<ipython-input-46-d9cc5e62cc22>:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  municipios['centroide'] = municipios.geometry.centroid


### Processo de krigagem

In [ ]:
datas = np.sort(df_estacoes['Data'].dropna().unique())

# Lista para armazenar os resultados
resultados_lista = []

for data in datas:
    # Filtrar os dados do dia
    df_dia = df_estacoes[df_estacoes['Data'] == data]

    # -----------------------------
    # Krigagem para Temperatura Média
    # -----------------------------
    df_temp = df_dia.dropna(subset=['temperatura_media', 'VL_LATITUDE', 'VL_LONGITUDE'])
    if len(df_temp) >= 3:
        x_temp = df_temp['VL_LONGITUDE'].values
        y_temp = df_temp['VL_LATITUDE'].values
        z_temp = df_temp['temperatura_media'].values
        try:
            OK_temp = OrdinaryKriging(
                x_temp, y_temp, z_temp,
                variogram_model='spherical',
                verbose=False,
                enable_plotting=False
            )
            z_temp_kriged, ss_temp = OK_temp.execute('points', municipios['lon'].values, municipios['lat'].values)
        except Exception as e:
            print(f"Erro na krigagem de temperatura em {data.date()}: {e}")
            z_temp_kriged = np.full(len(municipios), np.nan)
    else:
        z_temp_kriged = np.full(len(municipios), np.nan)

    # -----------------------------
    # Krigagem para Precipitação
    # -----------------------------
    df_prec = df_dia.dropna(subset=['preciptacao', 'VL_LATITUDE', 'VL_LONGITUDE'])
    if len(df_prec) >= 3:
        x_prec = df_prec['VL_LONGITUDE'].values
        y_prec = df_prec['VL_LATITUDE'].values
        z_prec = df_prec['preciptacao'].values
        try:
            # Se os valores forem constantes (sem variação), atribua o valor constante a todos os municípios
            if np.ptp(z_prec) == 0:
                z_prec_kriged = np.full(len(municipios), z_prec[0])
            else:
                OK_prec = OrdinaryKriging(
                    x_prec, y_prec, z_prec,
                    variogram_model='spherical',
                    verbose=False,
                    enable_plotting=False
                )
                z_prec_kriged, ss_prec = OK_prec.execute('points', municipios['lon'].values, municipios['lat'].values)
        except Exception as e:
            print(f"Erro na krigagem de precipitação em {pd.Timestamp(data).date()}: {e}")
            z_prec_kriged = np.full(len(municipios), np.nan)
    else:
        z_prec_kriged = np.full(len(municipios), np.nan)

    # -----------------------------
    # Krigagem para Umidade Média
    # -----------------------------
    df_umid = df_dia.dropna(subset=['umidade_media', 'VL_LATITUDE', 'VL_LONGITUDE'])
    if len(df_umid) >= 3:
        x_umid = df_umid['VL_LONGITUDE'].values
        y_umid = df_umid['VL_LATITUDE'].values
        z_umid = df_umid['umidade_media'].values
        try:
            OK_umid = OrdinaryKriging(
                x_umid, y_umid, z_umid,
                variogram_model='spherical',
                verbose=False,
                enable_plotting=False
            )
            z_umid_kriged, ss_umid = OK_umid.execute('points', municipios['lon'].values, municipios['lat'].values)
        except Exception as e:
            print(f"Erro na krigagem de umidade em {data.date()}: {e}")
            z_umid_kriged = np.full(len(municipios), np.nan)
    else:
        z_umid_kriged = np.full(len(municipios), np.nan)

    # -----------------------------
    # 5. Criar um DataFrame com os resultados do dia
    # -----------------------------
    df_resultado_dia = pd.DataFrame({
        'Data': data,
        'Municipio': municipios['CD_MUN'],
        'Temperatura_Media': z_temp_kriged,
        'Precipitacao': z_prec_kriged,
        'Umidade_Media': z_umid_kriged
    })

    resultados_lista.append(df_resultado_dia)

final_df = pd.concat(resultados_lista, ignore_index=True)

### Verificação

In [ ]:
final_df.head()

,Data,Municipio,Temperatura_Media,Precipitacao,Umidade_Media
0,2016-01-01,3100104,24.814255,2.332133,71.214895
1,2016-01-01,3100203,26.327830,-0.439036,63.004710
2,2016-01-01,3100302,25.058005,1.346237,69.315929
3,2016-01-01,3100401,24.156653,4.334769,74.274005
4,2016-01-01,3100500,27.890152,-0.146384,62.068986


In [ ]:
df_dia = df_estacoes[df_estacoes['Data'] == np.datetime64('2016-01-01')]
print("Registros com temperatura:", df_dia['temperatura_media'].dropna().shape[0])
print("Registros com precipitação:", df_dia['preciptacao'].dropna().shape[0])
print("Registros com umidade:", df_dia['umidade_media'].dropna().shape[0])

Registros com temperatura: 53
Registros com precipitação: 56
Registros com umidade: 53


In [ ]:
df_dia = df_estacoes[df_estacoes['Data'] == np.datetime64('2018-12-31')]
print("Registros com temperatura:", df_dia['temperatura_media'].dropna().shape[0])
print("Registros com precipitação:", df_dia['preciptacao'].dropna().shape[0])
print("Registros com umidade:", df_dia['umidade_media'].dropna().shape[0])

Registros com temperatura: 66
Registros com precipitação: 67
Registros com umidade: 66


### Exportação

In [ ]:
final_df.to_csv('resultado_krigagem_2016_2018.csv', index=False)

files.download('resultado_krigagem_2016_2018.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>